In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 用户配置区 ================================================
CSV_PATH = "your_data.csv"         # CSV文件路径
TIME_COLUMN = None                 # 手动指定时间列名（如'date'），None则自动检测
USE_SUBPLOTS = True                # 是否使用子图单独显示每个维度
STYLE = "whitegrid"                # 图表样式（whitegrid/darkgrid/ticks等）
FIG_SIZE = (12, 6)                 # 基础图表尺寸（子图模式下高度自动扩展）
# =========================================================

In [ ]:
try:
    # 读取CSV文件
    df = pd.read_csv(CSV_PATH)

    # 自动检测时间列（如果未手动指定）
    if TIME_COLUMN is None:
        time_candidates = ['date', 'time', 'datetime', 'timestamp']
        detected = [col for col in df.columns if col.lower() in time_candidates]
        TIME_COLUMN = detected[0] if detected else None

    # 处理时间列
    x_label = "时间"
    if TIME_COLUMN:
        try:
            df[TIME_COLUMN] = pd.to_datetime(df[TIME_COLUMN])
            df.sort_values(TIME_COLUMN, inplace=True)  # 按时间排序
            x = df[TIME_COLUMN]
        except Exception as e:
            print(f"警告：时间列{TIME_COLUMN}格式无效，将使用索引作为时序轴（{e}）")
            TIME_COLUMN = None
            x = df.index
            x_label = "索引"
    else:
        x = df.index
        x_label = "索引"

    # 提取数值列（排除时间列）
    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    if TIME_COLUMN and TIME_COLUMN in numeric_cols:
        numeric_cols.remove(TIME_COLUMN)

    if not numeric_cols:
        raise ValueError("未找到数值型数据列，请检查数据格式")

    # 可视化设置
    sns.set_style(STYLE)
    subplot_count = len(numeric_cols)
    colors = sns.color_palette("husl", subplot_count)

    # 绘制图表
    if USE_SUBPLOTS and subplot_count > 1:
        fig, axes = plt.subplots(
            subplot_count, 1,
            figsize=(FIG_SIZE[0], FIG_SIZE[1] * subplot_count),
            sharex=True
        )
        for i, (col, color) in enumerate(zip(numeric_cols, colors)):
            ax = axes[i]
            ax.plot(x, df[col], color=color)
            ax.set_ylabel(col, fontsize=10)
            if i == subplot_count - 1:
                ax.set_xlabel(x_label, fontsize=10)
            ax.grid(True, alpha=0.3)
        plt.suptitle("时序数据多维分析", y=0.92)
    else:
        plt.figure(figsize=FIG_SIZE)
        for col, color in zip(numeric_cols, colors):
            plt.plot(x, df[col], label=col, color=color)
        plt.xlabel(x_label)
        plt.ylabel("数值")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.title("时序数据综合分析")
        plt.tight_layout()

    plt.show()

In [ ]:
except FileNotFoundError:
    print(f"错误：文件 {CSV_PATH} 不存在")
except pd.errors.ParserError:
    print("错误：CSV文件解析失败，请检查文件格式")
except ValueError as ve:
    print(ve)
except Exception as e:
    print(f"未知错误：{str(e)}")